## Motion Planning through KOMO ##

In [1]:
import robotic as ry
import numpy as np
import time

## Part a) Adding a constraint to simulation

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

First, solve the task without collision constraint

In [5]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.025937, evals: 15, done: 1, feasible: 1, sos: 1.91685, f: 0, ineq: 0, eq: 0.00143336 }
[[-3.72973568e-02 -4.52759362e-01 -3.78892812e-02 -2.00578896e+00
  -8.91460284e-03  2.01392107e+00 -4.99904269e-01]
 [-7.46356766e-02 -4.05467234e-01 -7.58150761e-02 -2.01157957e+00
  -1.78331427e-02  2.02785432e+00 -4.99808708e-01]
 [-1.12055875e-01 -3.58072289e-01 -1.13813954e-01 -2.01737361e+00
  -2.67596697e-02  2.04181183e+00 -4.99713488e-01]
 [-1.49598778e-01 -3.10523521e-01 -1.51922596e-01 -2.02317310e+00
  -3.56984599e-02  2.05580552e+00 -4.99618774e-01]
 [-1.87305086e-01 -2.62770366e-01 -1.90177862e-01 -2.02898040e+00
  -4.46541256e-02  2.06984702e+00 -4.99524731e-01]
 [-2.25215359e-01 -2.14762799e-01 -2.28616860e-01 -2.03479830e+00
  -5.36317229e-02  2.08394772e+00 -4.99431515e-01]
 [-2.63370008e-01 -1.66451384e-01 -2.67277031e-01 -2.04063011e+00
  -6.26368552e-02  2.09811865e+00 -4.99339279e-01]
 [-3.01809315e-01 -1.17787274e-01 -3.06196237e-01 -2.04647964e+00
  -7.16757696e-02 

In [6]:
del komo
del C

Then, we add collision constraint with "accumulatedCollisions" feature

In [7]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [12]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e-1z])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.16484, evals: 78, done: 1, feasible: 0, sos: 2.50177, f: 0, ineq: 0, eq: 2.18668 }
[[-3.43529718e-02 -4.59909412e-01 -3.78989475e-02 -2.00939682e+00
  -9.20643165e-03  2.01013981e+00 -4.99788114e-01]
 [-6.87947766e-02 -4.19620627e-01 -7.58751582e-02 -2.01873877e+00
  -1.83893051e-02  2.02037510e+00 -4.99573667e-01]
 [-1.03412233e-01 -3.78939111e-01 -1.14003602e-01 -2.02797112e+00
  -2.75240277e-02  2.03079961e+00 -4.99354127e-01]
 [-1.38288145e-01 -3.37677632e-01 -1.52354680e-01 -2.03703967e+00
  -3.65841029e-02  2.04150362e+00 -4.99127034e-01]
 [-1.73499745e-01 -2.95659476e-01 -1.90992423e-01 -2.04589095e+00
  -4.55404990e-02  2.05257239e+00 -4.98889912e-01]
 [-2.09117397e-01 -2.52721013e-01 -2.29973293e-01 -2.05447263e+00
  -5.43612148e-02  2.06408487e+00 -4.98640192e-01]
 [-2.45203943e-01 -2.08713380e-01 -2.69345767e-01 -2.06273391e+00
  -6.30112163e-02  2.07611272e+00 -4.98375310e-01]
 [-2.81814823e-01 -1.63502907e-01 -3.09150892e-01 -2.07062620e+00
  -7.14529978e-02  2.0

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [13]:
del komo
del C

In [14]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.0])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [ ]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

## Possible State Solution by RRT

In [ ]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

In [ ]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

In [35]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [36]:
del komo
del rrt
del C

Now, let us give a state which is impossible for robot arm to reach.

##  Part c) Impossible State Solution by KOMO

In [37]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [ ]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

## Impossible State Solution by RRT

In [ ]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

In [ ]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

So we will get an error since there is no feasible solution

In [ ]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [41]:
# https://marctoussaint.github.io/robotics-course/tutorials.html -> Tutorials
# https://github.com/MarcToussaint/rai-tutorials/tree/main -> Tutorials
# https://marctoussaint.github.io/robotics-course/rai.html -> Documentation